In [1]:
import numpy as np
import matplotlib.pyplot as plt

def quantize(signal, step_size):
    return step_size * np.round(signal / step_size)

def compress_signal(signal, step_size):
    # Apply FFT to transform to frequency domain
    fft_coeffs = np.fft.fft(signal)
    
    # Apply logarithmic scaling
    log_scaled_coeffs = np.sign(fft_coeffs) * np.log1p(np.abs(fft_coeffs))
    
    # Quantize the scaled coefficients
    quantized_coeffs = quantize(log_scaled_coeffs, step_size)
    
    return quantized_coeffs

def decompress_signal(quantized_coeffs, step_size):
    # Apply inverse quantization
    log_scaled_coeffs = quantized_coeffs
    
    # Apply inverse logarithmic scaling
    fft_coeffs = np.sign(log_scaled_coeffs) * (np.expm1(np.abs(log_scaled_coeffs)))
    
    # Apply inverse FFT to transform back to time domain
    reconstructed_signal = np.fft.ifft(fft_coeffs)
    
    return np.real(reconstructed_signal)

# Example usage
if __name__ == "__main__":
    # Create a sample signal (e.g., a sine wave)
    t = np.linspace(0, 1, 500, endpoint=False)
    original_signal = np.sin(2 * np.pi * 5 * t)
    
    # Add some noise to the signal
    noisy_signal = original_signal + 0.5 * np.random.normal(size=t.shape)
    
    # Define quantization step size
    step_size = 0.1
    
    # Compress the signal
    compressed_signal = compress_signal(noisy_signal, step_size)
    
    # Decompress the signal
    reconstructed_signal = decompress_signal(compressed_signal, step_size)
    
    # Plot the original, noisy, and reconstructed signals
    plt.figure(figsize=(15, 5))
    plt.plot(t, noisy_signal, label="Noisy Signal")
    plt.plot(t, reconstructed_signal, label="Reconstructed Signal", linestyle='dashed')
    plt.legend()
    plt.show()

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [6]:
import numpy as np
from scipy.fftpack import dct, idct

def dct_transform(x):
    return dct(dct(x, axis=-1, norm='ortho'), axis=-2, norm='ortho')

def inverse_dct_transform(x):
    return idct(idct(x, axis=-1, norm='ortho'), axis=-2, norm='ortho')

def quantize(x, precision):
    scale = 2 ** (precision - 1) - 1
    return np.round(x * scale) / scale

def fdq(weights, precision_levels):
    # Apply DCT to transform weights to the frequency domain
    weights_f = dct_transform(weights)
    
    # Calculate importance as the magnitude of the frequency components
    importance = np.abs(weights_f)
    
    # Assign precision based on importance
    mean_importance = np.mean(importance)
    if mean_importance > 0.1:
        precision = precision_levels[0]  # 8-bit
    elif mean_importance > 0.01:
        precision = precision_levels[1]  # 4-bit
    else:
        precision = precision_levels[2]  # 2-bit
    
    # Quantize frequency components
    enc = quantize(weights_f, precision)
    enc = enc.astype(np.float16)

    
    # Apply inverse DCT to transform quantized weights back to the spatial domain
    weights_quantized = inverse_dct_transform(enc)
    
    return enc, weights_quantized

# Example weights
data = np.random.randn(2, 1, 5, 5)

# Precision levels (8-bit, 4-bit, 2-bit)
precision_levels = [8, 4, 2]

# Apply FDQ
enc, dec = fdq(data, precision_levels)


from sklearn.metrics import mean_squared_error as mse


print("Original Weights:")
print(data)
print("\nQuantized Weights:")
print(dec)
print("mse: ",mse(dec.flatten(), data.flatten()))



import sys
print(sys.getsizeof(data))
print(sys.getsizeof(dec))

print(sys.getsizeof(enc))

Original Weights:
[[[[-0.27669353  0.48177007  0.03384665  1.03789763  0.43715492]
   [-0.23605261 -0.15799487 -0.93510586 -0.07145706  1.1148076 ]
   [ 0.63546223 -0.33217822 -1.08045576  0.65972045 -1.06128466]
   [ 0.01446434 -0.71403993  0.99889664 -0.29100421 -0.74902595]
   [-0.02148173 -0.10872319 -0.86977533 -0.35646771  0.27552279]]]


 [[[ 0.46286088 -1.71835135 -0.95609272  0.23490862  0.1354916 ]
   [-1.68892451 -2.02730973  0.84900851 -0.48451502  1.10795943]
   [-0.87096969 -0.18579431 -0.44873071 -0.27163689 -0.809725  ]
   [ 0.39279619  0.05461368  0.29908946  1.17710088 -0.18872093]
   [-1.79479051  0.28408116  0.11637371  0.54058855 -1.6659671 ]]]]

Quantized Weights:
[[[[-0.27381733  0.47923997  0.0314666   1.0328925   0.44000664]
   [-0.23925811 -0.15803966 -0.93427694 -0.06981337  1.1154636 ]
   [ 0.6377116  -0.32916912 -1.0804163   0.6599506  -1.0604023 ]
   [ 0.01355999 -0.71755403  0.99780583 -0.29230633 -0.75086486]
   [-0.02070933 -0.11201125 -0.86664915 -0.35

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [18]:
from scipy.fftpack import dct, idct

class customFreq():
    def __init__(self, precision_levels = [8, 4, 2]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-2, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-2, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

    
data = list(np.random.rand(1,16))
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)


[[0.53604406 0.0610901  0.9221441  0.16179116 0.23270804 0.35519463
  0.94239974 0.3132268  0.33660358 0.9405134  0.47207183 0.90899837
  0.23666447 0.00627296 0.181839   0.17067395]]
Encoded [[ 1.693    0.1812  -0.496    0.3938  -0.00787 -0.07874  0.2441  -0.441
  -0.2441   0.0551   0.37     0.504   -0.1497   0.2913   0.6143   0.0551 ]]
After Quantization:  [[5.3436279e-01 6.1150607e-02 9.2041105e-01 1.6192099e-01 2.3348960e-01
  3.5325769e-01 9.4457871e-01 3.1339684e-01 3.3594674e-01 9.4315422e-01
  4.7185650e-01 9.0950066e-01 2.3558901e-01 6.9805980e-04 1.7989314e-01
  1.7423050e-01]]
<class 'numpy.uint16'> 17.391304347826086% smaller 
enc shape:  (1, 16)
dec shape:  (1, 16)


In [16]:

# Example weights
data = np.random.randn(1, 16)

# Precision levels (8-bit, 4-bit, 2-bit)
precision_levels = [8, 4, 2]
enc = cmp.encode(data)
# Apply FDQ
dec = cmp.decode(enc)


from sklearn.metrics import mean_squared_error as mse


print("Original Weights:")
print(data)
print("\nQuantized Weights:")
print(dec)
print("mse: ",mse(dec.flatten(), data.flatten()))



import sys
print(sys.getsizeof(data))
print(sys.getsizeof(dec))

print(sys.getsizeof(enc))

Original Weights:
[[ 0.49996309 -0.13236434  2.14822299 -1.49587808  0.43404989 -1.82781938
  -1.06790416  0.12624729  3.7489454  -0.53143777 -0.61384816 -0.8358371
  -1.41729724 -1.00901338 -0.23867723 -0.95665989]]

Quantized Weights:
[[ 0.49456245 -0.13293028  2.1461558  -1.5002595   0.4361813  -1.8316177
  -1.0698416   0.12771547  3.749366   -0.5278095  -0.6143816  -0.8368098
  -1.4176418  -1.0082784  -0.23668753 -0.9593642 ]]
mse:  6.521250362708415e-06
248
184
152


In [17]:
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")


<class 'numpy.uint16'> 17.391304347826086% smaller 
